In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import json
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import cross_val_score

with open('sensor_data.json', 'r') as f:
    data = json.load(f)

df = pd.DataFrame(data)

X = df[['acceleration', 'rotation', 'magnetic_field', 'light']]
y = df['sos_triggered']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_scaled, y_train)

y_pred = rf_model.predict(X_test_scaled)

# print("Classification Report:")
# print(classification_report(y_test, y_pred))

cv_scores = cross_val_score(rf_model, X_train_scaled, y_train, cv=5)
# print("\nCross-validation scores:", cv_scores)
# print("Average CV score:", cv_scores.mean())

feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

# print("\nFeature Importance:")
# print(feature_importance)

# plt.figure(figsize=(8, 6))
# cm = confusion_matrix(y_test, y_pred)
# sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
# plt.title('Confusion Matrix')
# plt.ylabel('True Label')
# plt.xlabel('Predicted Label')
# plt.show()

def predict_sos(acceleration, rotation, magnetic_field, light):
    new_data = np.array([[acceleration, rotation, magnetic_field, light]])
    new_data_scaled = scaler.transform(new_data)
    prediction = rf_model.predict(new_data_scaled)
    probability = rf_model.predict_proba(new_data_scaled)[0]
    return {
        'prediction': int(prediction[0]),
        'probability': float(probability[prediction[0]])
    }

# Example predictions
test_cases = [
    {'acceleration': 125.80, 'rotation': 18, 'magnetic_field': 48.97, 'light': 200},  # Normal case
    {'acceleration': 130, 'rotation': 15, 'magnetic_field': 46, 'light': 305}  # Anomalous case
]

print("\nTest Predictions:")
for case in test_cases:
    result = predict_sos(case['acceleration'], case['rotation'],
                        case['magnetic_field'], case['light'])
    print(f"\nInput: {case}")
    print(f"Prediction: {'true' if result['prediction'] == 1 else 'false'}")
    print(f"Confidence: {result['probability']:.2%}")


Test Predictions:

Input: {'acceleration': 125.8, 'rotation': 18, 'magnetic_field': 48.97, 'light': 200}
Prediction: true
Confidence: 62.00%

Input: {'acceleration': 130, 'rotation': 15, 'magnetic_field': 46, 'light': 305}
Prediction: true
Confidence: 60.00%


c:\Users\rjzam\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\rjzam\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [7]:
import joblib
model = rf_model  # Replace with your trained model
joblib.dump(model, 'model.pkl')


['model.pkl']